<a href="https://colab.research.google.com/github/SerjoschDuering/cloneSpeckleStream/blob/main/cloneSpeckleStream_public.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Speckle Stream Cloning

This notebook demonstrates how to clone a Speckle stream from one to another. It uses the Speckle SDK to interact with a Speckle Server and perform operations like fetching branches and commits, and cloning them into a target stream.

## Requirements

- Speckle SDK: You can install it using pip (`pip install specklepy`)
- A Speckle account: You'll need an account to access a Speckle Server. You can create one [here](https://speckle.systems/).
- A Speckle Server URL and Personal Access Token: These are required to authenticate with the server.

## Usage

The main function for cloning a stream is `clone_stream()`, which takes the following arguments:

- `source_stream`: The URL of the stream that you want to clone.
- `target_stream`: The URL of the stream where you want to clone the source stream.
- `server_url`: The URL of your Speckle Server (optional if you're using the default account stored on your machine).
- `token`: Your Personal Access Token for the Speckle Server (optional if you're using the default account stored on your machine).
- `max_commits`: The maximum number of commits that you want to clone from each branch (default is 1).
- `max_branches`: The maximum number of branches the script iterates through (default is 100)

Here's how you can use the function:

```python
SERVER = 'your_server_url'
TOKEN = 'your_token'
source_stream = 'source_stream_url'
target_stream = 'target_stream_url'

clone_stream(source_stream, target_stream, max_commits=1, max_branches=100, server_url=SERVER, token=TOKEN)


In [2]:
!pip install specklepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.9/102.9 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.9/202.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.3 MB/s eta 0:00:00
  Created wheel for stringcase: filename=stringcase-1.2.0-py3-none-any.whl size=3568 sha256=e198f209378f00aff48bba1993178716cb3eab5b80b15b8a70e4fabe4ffe34fe
  Stored in directory: /root/.cache/pip/wheels/31/ba/22/1a2d952a9ce8aa86e42fda41e2c87fdaf20e238c88bf8df013
Successfully built stringcase


In [3]:
from specklepy.api.client import SpeckleClient
from specklepy.api.credentials import get_default_account
from specklepy.transports.server import ServerTransport
from specklepy.api import operations

In [5]:
def clone_stream(source_stream, target_stream, server_url=None, token=None, max_commits=1, max_branches=100):

    if server_url and token:
        client = SpeckleClient(host=server_url)
        client.authenticate_with_token(token=token)
    else:
        # Try to get default account
        account = get_default_account()
        client = SpeckleClient(host=account.serverUrl)
        client.authenticate(token=account.token)

    source_stream_id = source_stream.split('/')[-1]
    target_stream_id = target_stream.split('/')[-1]

    branches = client.branch.list(source_stream_id, max_branches)


    print("branches found on stream:")
    [print(n.name) for n in branches]

    # Fetch source stream transport
    source_transport = ServerTransport(client=client, stream_id=source_stream_id)

    # Fetch target stream transport
    target_transport = ServerTransport(client=client, stream_id=target_stream_id)

    report = {}

    # Iterate through each branch
    for branch in branches:
        branch_name = branch.name
        report[branch_name] = {}

        # Create branch in target stream
        client.branch.create(target_stream_id, branch_name, branch.description)

        # Fetch all commits in current branch
        try:
            commits = client.branch.get(source_stream_id, branch_name).commits.items
        except Exception as e:
            report[branch_name]['error'] = f'Failure to fetch commits: {str(e)}'
            continue

        # If branch has fewer commits than max_commits, clone only the available ones
        num_commits = min(len(commits), max_commits)

        # Iterate through each commit in the branch
        for i in range(num_commits):
            commit = commits[i]
            print("processing commit: ",commit )
            # Fetch commit objects from source stream
            commit = client.commit.get(source_stream_id, commit.id)
            objects = operations.receive(commit.referencedObject, source_transport)

            # Send commit objects to target stream
            try:
                new_id = operations.send(objects, [target_transport])
                client.commit.create(
                    stream_id=target_stream_id,
                    branch_name=branch_name,
                    object_id=new_id,
                    message=commit.message,
                )
                report[branch_name][commit.id] = 'Success'
            except Exception as e:
                report[branch_name][commit.id] = f'Failure: {str(e)}'

    # Print report
    for branch_name, commits in report.items():
        print(f'Branch: {branch_name}')
        for commit_id, status in commits.items():
            print(f'\tCommit: {commit_id}, Status: {status}')


In [6]:
source_stream = "soruce stream url here"
target_stream = "target stream url here"

TOKEN = None  # if None, script tries to use get_default_account()
SERVER = None # if None, script tries to use get_default_account()

In [ ]:
clone_stream(source_stream, target_stream, max_commits=1, max_branches=100, server_url=SERVER, token=TOKEN)
